In [32]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from string import punctuation
import spacy
from spacy import displacy
from SPARQLWrapper import SPARQLWrapper2, JSON
import pandas as pd
from tabulate import tabulate
import tkinter as tk
from tkinter.scrolledtext import ScrolledText


In [13]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nlp_spacy = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /Users/changan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/changan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/changan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/changan/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [14]:
questions = [
    "Could you provide a list of highest rated and popular Multiplayer feature game titles, along with their prices, that requires memory 8 GB RAM?",
    "Give me a list of game titles that price below $50.",
    "What are the titles,release date of the highest rated and popular action genre game of all time?",
    "Can you recommend game titles that has theme World War 2, and price below $60 but not published by company Activision Inc?",
    "What are the titles and prices of the highest rated games published by company Activision Inc in 2023?"
]

In [15]:
def normalize_and_tokenize(question):
    # Normalize and tokenize the text
    text = question.lower()
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)
    return word_tokenize(text)

def pos_tagging(tokens):
    return pos_tag(tokens)

def map_to_ontology(tagged_tokens, class_keywords, object_property_keywords, data_property_keywords):
    lemmatizer = WordNetLemmatizer()
    classes = []
    object_properties = []
    data_properties = []
    for word, tag in tagged_tokens:
        word_lemmatized = lemmatizer.lemmatize(word)

        
        if word_lemmatized in class_keywords:
            classes.append(class_keywords[word_lemmatized])

       
        if word_lemmatized in object_property_keywords:
            object_properties.append(object_property_keywords[word_lemmatized])
        if word_lemmatized in data_property_keywords:
            data_properties.append(data_property_keywords[word_lemmatized])
    return classes,object_properties,data_properties
def get_individuals_from_question(question):
    doc = nlp_spacy(question)
    individuals = {}

    
    genre_keywords = ["Action", "Adventure", "Strategy", "RPG", "Shooter"]  

    temp_ram_value = ""
    for ent in doc.ents:
        
        if ent.label_ == 'CARDINAL' and not temp_ram_value:
            temp_ram_value = ent.text
            continue
        if ent.text == 'GB' and temp_ram_value:
            individuals[f"{temp_ram_value} {ent.text}"] = 'RAM'  
            temp_ram_value = ""
            continue

        
        if ent.text == 'RAM':
            continue

       
        if ent.label_ == 'MONEY':
            key = ent.text.lstrip('$')
            if "below" in question.lower():
                key = key.split()[-1] 
       
        elif ent.label_ == 'ORG' and ent.text.endswith('Inc'):
            key = ent.text[:-4] 
        else:
            key = ent.text

        
        if key in individual_keywords:
            mapped_key = individual_keywords[key]
        else:
            mapped_key = key

        individuals[mapped_key] = ent.label_
    for genre in genre_keywords + ["Multiplayer"]:
        if genre.lower() in question.lower():
            category = "Genre" if genre != "Multiplayer" else "Feature"
            individuals[genre] = category

    return individuals


In [16]:
class_keywords = {
    'game': 'Game', 'genre': 'Genre', 'company': 'PublishingCompany', 'character': 'Character', 'price': 'GamePrice',
    'costs': 'GamePrice','feature': 'Game_Features', 'theme':'Theme', 'memory':'RAM',
    'rated':'Ratings', 'rating':'Ratings', 'ratings':'Ratings'
}
object_property_keywords = {
    'price': 'has_price',
    'cost': 'has_price',  
    'rating': 'gets_ratings',
    'ratings': 'gets_ratings',
    'rated': 'gets_ratings',
    'score': 'gets_ratings',  
    'feature': 'has_features',
    'features': 'has_features',  
    'theme': 'has_theme',
    'themes': 'has_theme',  
    'genre': 'has_Genre',
    'genres': 'has_genre',  
    'company': 'publishes_Game', 
    'memory': 'requires_RAM'  
}
data_property_keywords = {
    'price': 'Game_Price_Value',
    'cost': 'Game_Price_Value', 
    'rating': 'Ratings_Value',
    'Game': 'Game_Title',
    'Games': 'Release_Date',
    'game': 'Release_Date',
    'games': 'Release_Date',
    'ratings': 'Ratings_Value',
    'rated': 'Ratings_Value',
    'score': 'Ratings_Value',  
    'feature': 'feature_type',
    'features': 'feature_type',  
    'theme': 'themeType',
    'themes': 'ThemeType',  
    'genre': 'Genre_type',
    'genres': 'Genre_type', 
    'company': 'Name', 
    'memory': 'capacity'  
}
individual_keywords = {
    'Multiplayer': 'Multiplayer', 
    'multiplayer': 'Multiplayer',
    '$50': '50',  
    '$60': '60'  
}

In [17]:
def construct_query(question):
    individuals = get_individuals_from_question(question)
    tokens = normalize_and_tokenize(question)
    tagged_tokens = pos_tagging(tokens)
    classes, object_properties, data_properties = map_to_ontology(tagged_tokens, class_keywords, object_property_keywords, data_property_keywords)
    query_prefix = "PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX Game: <http://www.gamemania.com#>\n"
    
    # Start constructing the query
    select_clause = "SELECT"
    where_clause = "WHERE {\n"
    filter_clauses = []
    order_by_clause = ""
    limit_clause=""
    # Handling Classes
    if 'Game' in classes:
        select_clause += " ?GameTitle"
        select_clause += " ?ReleaseDate"
        where_clause += "  ?game a Game:Game .\n  ?game Game:GameTitle ?GameTitle .\n  ?game Game:Release_date ?ReleaseDate. \n"
    if 'GamePrice' in classes:
        select_clause += " ?GamePrice"
    if 'Ratings' in classes:
        select_clause += " ?Ratings"
    if 'RAM' in classes:
        select_clause += " ?RAM"
    # Handling Object Properties
    for prop in object_properties:
        if prop == 'has_price':
            where_clause += "  ?game Game:has_price ?price .\n"
        elif prop == 'gets_ratings':
            where_clause += "  ?game Game:gets_ratings ?ratings .\n"
        elif prop == 'has_features':
            where_clause += "  ?game Game:has_features ?features .\n"
        elif prop == 'requires_RAM':
            where_clause += "  ?game Game:requires_RAM ?ram .\n"
        elif prop == 'has_genre':
            where_clause += "  ?game Game:has_genre ?genre .\n"
        elif prop == 'has_theme':
            where_clause += "  ?game Game:has_theme ?theme .\n"
        elif prop == 'publishes_Game':
            where_clause += "  ?pubCompany Game:publishes_Game ?game .\n"

    # Handling Data Properties
    if 'Game_Price_Value' in data_properties:
        where_clause += "  ?price Game:Game_Price_Value ?GamePrice .\n"
    if 'Ratings_Value' in data_properties:
        where_clause += "  ?ratings Game:Ratings_Value ?Ratings .\n"
    if 'capacity' in data_properties:
        where_clause += "  ?ram Game:capacity ?RAM .\n"
    if 'Genre_type' in data_properties:
        select_clause += " ?GenreType"
        where_clause += "  ?genre Game:Genre_type ?GenreType .\n"
    if 'themeType' in data_properties:
        select_clause += " ?themeType"
        where_clause += "  ?theme Game:themeType ?themeType .\n"
    if 'feature_type' in data_properties:
        select_clause += " ?feature"
        where_clause += "  ?features Game:feature_type ?feature .\n"
    if 'Name' in data_properties:
        select_clause += " ?PublishingCompanyName"
        where_clause += "  ?pubCompany Game:Name ?PublishingCompanyName .\n"

    # Handling Individuals
    for individual, category in individuals.items():
        if category == 'Feature':
            filter_clauses.append(f"?feature = \"{individual}\"")
        elif category == 'RAM':
            filter_clauses.append(f"?RAM = \"{individual}\"")
        elif category == 'MONEY':
            filter_clauses.append(f"?GamePrice < \"{individual}\"")
        elif category == 'EVENT':
            filter_clauses.append(f"?themeType = \"{individual}\"")
        elif category == 'ORG':
    
            if 'not published by' in question:  
                filter_clauses.append(f"?PublishingCompanyName != \"{individual}\"")
            else:  
                filter_clauses.append(f"?PublishingCompanyName = \"{individual}\"")
            
        elif category == 'Genre':
            filter_clauses.append(f"?GenreType = \"{individual}\"")
        elif category == 'DATE':
            filter_clauses.append(f"?ReleaseDate = \"{individual}\"")
    if filter_clauses:
        where_clause += "  FILTER(" + " && ".join(filter_clauses) + ").}\n"
    if 'highest rated' in question.lower() or 'Ratings' in classes:
        order_by_clause = "ORDER BY DESC(?Ratings)"
        limit_clause = "LIMIT 5"
    query = f"{query_prefix}{select_clause}\n{where_clause}\n{order_by_clause}\n{limit_clause}"

    return query

In [18]:

for question in questions:
    tokens = normalize_and_tokenize(question)
    tagged_tokens = pos_tagging(tokens)
    classes, object_properties, data_properties = map_to_ontology(tagged_tokens, class_keywords, object_property_keywords, data_property_keywords)
    individuals = get_individuals_from_question(question)
    print(f"Question: {question}")
    print(f"Extracted Individuals: {individuals}\n")
    print(f"Extracted Classes: {classes}")
    print(f"Extracted object Properties: {object_properties}\n")
    print(f"Extracted data Properties: {data_properties}\n")

Question: Could you provide a list of highest rated and popular Multiplayer feature game titles, along with their prices, that requires memory 8 GB RAM?
Extracted Individuals: {'Multiplayer': 'Feature', '8 GB': 'RAM'}

Extracted Classes: ['Ratings', 'Game_Features', 'Game', 'GamePrice', 'RAM']
Extracted object Properties: ['gets_ratings', 'has_features', 'has_price', 'requires_RAM']

Extracted data Properties: ['Ratings_Value', 'feature_type', 'Release_Date', 'Game_Price_Value', 'capacity']

Question: Give me a list of game titles that price below $50.
Extracted Individuals: {'50': 'MONEY'}

Extracted Classes: ['Game', 'GamePrice']
Extracted object Properties: ['has_price']

Extracted data Properties: ['Release_Date', 'Game_Price_Value']

Question: What are the titles,release date of the highest rated and popular action genre game of all time?
Extracted Individuals: {'Action': 'Genre'}

Extracted Classes: ['Ratings', 'Genre', 'Game']
Extracted object Properties: ['gets_ratings', 'has_G

In [19]:
# for question in questions:
#     sparql_query = construct_query(question)
#     print(f"Question: {question}\nGenerated SPARQL Query:\n{sparql_query}\n")

In [33]:
def exec_query(query):
    sparql = SPARQLWrapper2("http://localhost:3030/GameEngine/query")
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)

    try:
        response = sparql.query().bindings
        data = [{str(key): str(value.value) for key, value in result.items()} for result in response]

        pd.set_option('display.max_colwidth', 350)  
        pd.set_option('display.expand_frame_repr', True) 
        df = pd.DataFrame(data)
        formatted_table = tabulate(df, headers='keys', tablefmt='pretty', showindex=False)
        return formatted_table
    except Exception as e:
        return f"An error occurred: {str(e)}"

def run_query():
    question = query_entry.get()
    sparql_query = construct_query(question)
    result = exec_query(sparql_query)
    output_box.delete('1.0', tk.END)
    output_box.insert(tk.END, result)

root = tk.Tk()
root.title("Game Recommendation Engine")

query_entry = tk.Entry(root, width=70, font=('', 16))
query_entry.pack(padx=10, pady=10)

run_button = tk.Button(root, text="Generate", command=run_query)
run_button.pack(pady=5)

output_box = ScrolledText(root, height=40, width=100, font=('',16))
output_box.pack(padx=10, pady=10)

root.mainloop()
